# 编译器和解释器
:label:`sec_hybridize`

到目前为止，本书主要关注的是命令式编程，它使用诸如 `print`、`+` 和 `if` 之类的语句来改变程序的状态。请考虑以下简单命令式程序的例子。

In [1]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10


Python是一种*解释型语言*。在评估上述`fancy_func`函数时，它会按顺序执行构成函数体的操作。也就是说，它会先计算`e = add(a, b)`并将结果存储为变量`e`，从而改变程序的状态。接下来的两条语句`f = add(c, d)`和`g = add(e, f)`也将以类似的方式执行，进行加法运算并将结果存储为变量。:numref:`fig_compute_graph`展示了数据流。

![命令式程序中的数据流。](../img/computegraph.svg)
:label:`fig_compute_graph`

尽管命令式编程很方便，但它可能效率低下。一方面，即使在整个`fancy_func`中多次调用`add`函数，Python也会单独执行这三个函数调用。如果这些操作是在GPU（甚至多个GPU）上执行的，那么由Python解释器引起的开销可能会变得非常大。此外，直到`fancy_func`中的所有语句都执行完毕之前，它都需要保存变量`e`和`f`的值。这是因为我们不知道在执行了语句`e = add(a, b)`和`f = add(c, d)`之后，程序的其他部分是否会使用到变量`e`和`f`。

## 符号编程

考虑另一种选择，*符号编程*，其中通常只有在过程完全定义后才会执行计算。这一策略被包括Theano和TensorFlow在内的多个深度学习框架所采用（后者已获得了命令式扩展）。它通常涉及以下步骤：

1. 定义要执行的操作。
2. 将这些操作编译成可执行程序。
3. 提供所需的输入并调用编译后的程序来执行。

这允许大量的优化。首先，在很多情况下我们可以跳过Python解释器，从而消除一个性能瓶颈，这个瓶颈在多块快速GPU与单个CPU上的Python线程配对时可能会变得非常明显。其次，编译器可以优化并重写上述代码为`print((1 + 2) + (3 + 4))`甚至`print(10)`。这是可能的，因为编译器可以在将其转换为机器指令之前看到完整的代码。例如，当不再需要某个变量时，它可以释放内存（或根本不分配）。或者它可以将代码完全转换为等价的形式。
为了更好地理解，请考虑下面对命令式编程的模拟（毕竟它是Python）。

In [2]:
def add_():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))
10


命令式（解释型）编程和符号式编程之间的差异如下：

* 命令式编程更简单。当在Python中使用命令式编程时，大部分代码都是直接且易于编写的。调试命令式编程代码也更容易。这是因为更容易获取并打印所有相关的中间变量值，或者使用Python的内置调试工具。
* 符号式编程更高效且更容易移植。符号式编程使得在编译期间优化代码更加容易，同时还能将程序转换成与Python无关的格式。这使得程序可以在非Python环境中运行，从而避免了任何可能与Python解释器相关的性能问题。

## 混合编程

历史上大多数深度学习框架会在命令式或符号式方法之间做出选择。例如，Theano、TensorFlow（受前者启发）、Keras和CNTK以符号形式构建模型。相反，Chainer和PyTorch采取了命令式方法。TensorFlow 2.0和后来版本的Keras中添加了命令式模式。

如上所述，PyTorch 基于命令式编程并使用动态计算图。为了利用符号编程的可移植性和效率，开发者考虑是否可以结合这两种编程范式的优点。这导致了 torchscript 的产生，它允许用户使用纯命令式编程进行开发和调试，同时能够将大多数程序转换为符号程序，以便在需要产品级计算性能和部署时运行。

## 混合`Sequential`类

要了解混合化是如何工作的，最简单的方法是考虑具有多层的深度网络。传统上，Python解释器需要执行所有层的代码以生成可以转发到CPU或GPU的指令。对于单个（快速）计算设备来说，这不会引起任何重大问题。另一方面，如果我们使用像AWS P3dn.24xlarge实例这样的高级8-GPU服务器，Python将难以让所有GPU保持忙碌状态。单线程的Python解释器在这里成为了瓶颈。让我们看看如何通过用`HybridSequential`替换`Sequential`来解决代码中的大部分问题。我们首先定义一个简单的MLP。

In [3]:
import torch
from torch import nn
from d2l import torch as d2l


# Factory for networks
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[-0.1602,  0.0003]], grad_fn=<AddmmBackward0>)

通过使用 `torch.jit.script` 函数转换模型，我们能够编译和优化MLP中的计算。模型的计算结果保持不变。

In [4]:
net = torch.jit.script(net)
net(x)

tensor([[-0.1602,  0.0003]], grad_fn=<AddmmBackward0>)

这看起来简直好得令人难以置信：像以前一样编写代码，然后只需使用 `torch.jit.script` 转换模型。一旦完成这一步，网络就会被优化（我们将在下面进行性能基准测试）。

### 通过混合优化加速

为了展示编译带来的性能提升，我们将比较`net(x)`在混合优化前后的评估时间。首先定义一个类来测量这个时间。在整个章节中，当我们着手测量（并改进）性能时，这将非常有用。

In [5]:
#@save
class Benchmark:
    """For measuring running time."""
    def __init__(self, description='Done'):
        self.description = description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

现在我们可以调用网络两次，一次使用torchscript，一次不使用torchscript。

In [6]:
net = get_net()
with Benchmark('Without torchscript'):
    for i in range(1000): net(x)

net = torch.jit.script(net)
with Benchmark('With torchscript'):
    for i in range(1000): net(x)

Without torchscript: 2.1447 sec


With torchscript: 4.0545 sec


如上述结果所示，使用 `torch.jit.script` 函数将 `nn.Sequential` 实例进行脚本化后，通过符号编程提高了计算性能。

### 序列化

编译模型的一个好处是，我们可以将模型及其参数序列化（保存）到磁盘。这使我们能够以一种与所选前端语言无关的方式来存储模型。这样，我们可以将训练好的模型部署到其他设备上，并轻松使用其他前端编程语言。同时，代码通常比在命令式编程中所能实现的要快。让我们来看看`save`函数的实际应用。

In [7]:
net.save('my_mlp')
!ls -lh my_mlp*

-rw-r--r-- 1 ci ci 651K Aug 18 19:32 my_mlp


## 摘要

* 命令式编程使得设计新模型变得容易，因为可以编写包含控制流的代码，并能够使用大量的Python软件生态系统。
* 符号式编程要求我们先指定程序并编译后再执行。这样做的好处是性能得到提升。

## 练习题

1. 复习前几章中你感兴趣的模型。你能通过重新实现它们来提高计算性能吗？

[讨论](https://discuss.d2l.ai/t/2490)